### TensorFlow Basics
#### Notebook author: Nirupam Purushothama

This is just a practice notebook. All of the code is from the reference book. Reference book GIT is [here](https://github.com/ageron/handson-ml/)

Reference book: Hands-on MachineLearning with Sci-kit and TensorFlow - Aurelien Geron

In [1]:
import tensorflow as tf

In [2]:
# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [3]:
x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
init = tf.global_variables_initializer()
f = x*x*y + y + 2

In [4]:
with tf.Session() as sess:
#    x.initializer.run()
#    y.initializer.run()
    init.run() # initialize all the variables
    result = f.eval()
    print(result)
# session is automatically closed at the end of the run

42


In [5]:
# Without "with session". You can use an interactive session
sess = tf.InteractiveSession()
init.run()
result = f.eval()
print(result)
sess.close()

# We will need to close the session by ourselves

42


In [6]:
# Running multiple graphs

x1 = tf.Variable(1) # gets added to the default graph
print(x1.graph is tf.get_default_graph())

# If want to create it under a different graph then
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)
    
print(x2.graph is graph)
print(x2.graph is tf.get_default_graph())

True
True
False


[tf.Constant vs. tf.Variable](https://groups.google.com/a/tensorflow.org/forum/#!topic/discuss/bodLatNWe7Q)

A variable can be assigned to, it's value can be changed. A constant is constant.

More subtly: A constant's value is stored in the graph and its value is replicated wherever the graph is loaded. A variable is stored separately, and may live on a parameter server. 

In [7]:
# Passing multi-dimensional arrays as inputs and obtaining an array as output
import numpy as np
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
m,n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m,1)), housing.data]

X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
x1 = tf.Variable(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name="y")
XT = tf.transpose(X)

# Regression's normal equation computation
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT,X)),XT),y)

with tf.Session() as sess:
    x1.initializer.run()
    theta_value = theta.eval()
    
print(theta_value)

[[ -3.63148041e+01]
 [  4.38327283e-01]
 [  9.54509154e-03]
 [ -1.09111905e-01]
 [  6.51177227e-01]
 [ -3.67034363e-06]
 [ -3.80339054e-03]
 [ -4.14844275e-01]
 [ -4.27279472e-01]]


In [8]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

In [11]:
# Finding the same thetas as above using gradient descent (using tensorflow)
reset_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name="y")

# n+1 x 1 array with all values in the range of [-1,1]
theta = tf.Variable(tf.random_uniform([n+1,1],-1.0,1.0),name="theta") 

y_pred = tf.matmul(X, theta, name="predictions")

error = y_pred - y

mse = tf.reduce_mean(tf.square(error), name="mse")

gradients = 2/m * tf.matmul(tf.transpose(X), error)

training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE: ", mse.eval())
        
        sess.run(training_op)
        
    best_theta = theta.eval()

Epoch 0 MSE:  12.408
Epoch 100 MSE:  0.755197
Epoch 200 MSE:  0.542087
Epoch 300 MSE:  0.53317
Epoch 400 MSE:  0.530538
Epoch 500 MSE:  0.528797
Epoch 600 MSE:  0.527548
Epoch 700 MSE:  0.52665
Epoch 800 MSE:  0.526001
Epoch 900 MSE:  0.525533


In [12]:
# Mini-batch gradient descent
reset_graph()

X = tf.placeholder(tf.float32, shape=(None, n+1), name = "X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

batch_size = 100
n_batches = int(np.ceil(m/batch_size))

theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

In [ ]:
def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)
    indices = np.random.randint(m, size=batch_size)
    X_batch = scaled_housing_data_plus_bias[indices]
    y_batch = housing.target.reshape(-1, 1)[indices]
    return X_batch, y_batch

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()

In [14]:
best_theta

array([[ 2.07144761],
       [ 0.84620124],
       [ 0.11558538],
       [-0.26835838],
       [ 0.32982787],
       [ 0.00608358],
       [ 0.07052909],
       [-0.87988573],
       [-0.86342508]], dtype=float32)